## Offline Setup

In [15]:
import requests

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

PROJECT_ID = 550
USERNAME = "Anonymous"
session = requests.session()

## Online Setup

In [16]:
main_response = session.get("https://mg1.gis-net.co.il/PardesHanaKarkurGis/")
assert main_response.ok

login_response = session.post("https://mg1.gis-net.co.il/PardesHanaKarkurApi/api/auth/UserLogin", json={
    "userName": USERNAME,
    "userPassword": "",
    "projId": PROJECT_ID,
})
assert login_response.ok

token = login_response.json()
assert isinstance(token, str)
auth = BearerAuth(token)

lut_response = session.get(f"https://mg1.gis-net.co.il/PardesHanaKarkurApi/api/app/GetLutValues?projId={PROJECT_ID}", auth=auth)
assert lut_response.ok
lookup_table = lut_response.json()

first_toc_response = session.post("https://mg1.gis-net.co.il/PardesHanaKarkurApi/api/map/FirstLoadingMap", auth=auth, json={
    "mapSession": "",
    "mapName": "",
    "projId": PROJECT_ID,
})
assert first_toc_response.ok
first_toc = first_toc_response.json()

toc_response = session.post("https://mg1.gis-net.co.il/PardesHanaKarkurApi/api/map/getMapToc", auth=auth, json={
    "mapSession": first_toc["sessionId"],
    "mapName": first_toc["mapName"],
    "projId": PROJECT_ID,
})
assert toc_response.ok
toc = toc_response.json()

assert first_toc == toc

## Download

In [ ]:
import random
from PIL import Image
from io import BytesIO

img_response = session.get(f"https://mg1.gis-net.co.il/mapguide{PROJECT_ID}/mapagent/mapagent.fcgi", params={
    "USERNAME": USERNAME,
    "SEQ": 0.9600439442289928,  # Change?
    "OPERATION": "GETDYNAMICMAPOVERLAYIMAGE",
    "VERSION": "2.0.0",
    "LOCALE": "en",
    "CLIENTAGENT": "ol.source.ImageMapGuide source",
    "CLIP": 1,
    "SETDISPLAYDPI": 96,
    "SETDISPLAYWIDTH": 1920,
    "SETDISPLAYHEIGHT": 1080,
    "SETVIEWSCALE": 50000.1000002,
    "SETVIEWCENTERX": 198199.77139954278,
    "SETVIEWCENTERY": 709005.376344086,
    "MAPNAME": "PardesHanaKarkurGIS",
    "FORMAT": "PNG",
    "SESSION": toc["sessionId"],
    "BEHAVIOR": 2,
    "unique": random.randint(100, 999),
    #hideLayers:,
    #showLayers:,
})
assert img_response.ok

image = Image.open(BytesIO(img_response.content))
display(image)
